# Chapter 4. The Cross-Entropy Method
 
#### DQN(Deep Q-Network), Advantage Actor-Critic 등과 같은 RL 기법 중 하나 
## * 장점 
#### - 간단함(simplicity)
- 직관적인 방법 
- Pytorch에서 100 줄 이내로 구현 가능 

#### - 좋은 수렴(Good convergence)
- 간단한 환경에서 잘 작동 
    - 복잡하고 여러 단계의 정책을 학습하도록 요구하지 않는 환경 
    - 적은 에피소드와 잦은 보상을 주는 환경 
    
- 모든 현실적인 문제들이 적합하지는 않지만 어떠한 환경에서는 잘 작동함 


#### 이번 장에서는..
- cross-entropy의 실제 사용되는 부분(practical side) 
- Gym의 CartPole 과 FrozenLake에서 cross-entropy가 작동하는 방법 
- cross-entropy의 이론적인 배경 

# <font color='blue'> Taxonomy of RL methods  <font>

#### 강화학습의 모든 방법들은 여러 방면에서 분류 가능 
#### 여러 방법들이 있지만 여기서는 3가지로 분류 


## * Model-free / Model-based 
#### - Model-free
- 환경이나 보상에 대한 모델을 생성하지 않음 
- 관찰(액션과 연관된 값)과 액션을 연결하는 작업만 수행 
- 에이전트는 현재 관찰에 대해 어떠한 연산을 한 뒤 수행할 액션을 선택함 
- 많은 관찰이 있는 복잡한 환경에서 좋은 모델을 생성하기 어려울 때 사용 


#### - Model-based
- 다음 관찰 또는 보상을 예측하고자 함 
- 예측을 기반으로 에이전트는 가능한 최고의 액션을 선택하려 함 
- 결정적인 환경(deterministic environments)에서 사용 
    - 엄격한 규칙이 존재하는 보드 게임 
    
    
최근, 두 방법의 장점을 함께 사용하는 연구가 활발히 진행 중 (17장) 
    


## * Value-based / Policy-based
#### - policy-based 
- 에이전트의 정책을 직접적으로 근사(approximating) 
- 정책은 step마다 에이전트가 수행해야 할 액션을 알려줌 
- 정책은 주로 가능한 액션의 확률분포로 표현 

#### - value-based
- 액션의 확률 대신, 에이전트는 가능한 모든 액션의 가치(value)를 계산하고 최고의 가치를 선택 

둘 다 유명한 방법, value-based는 part2, policy-based는 part3에서 다룸

## * On-policy / Off-policy
#### - on-policy 
- 과거 데이터를 사용하지 않는 방법 
- 학습하는 정책과 행동하는 정책이 반드시 같아야 함 


#### - off-policy 
- 과거 데이터 (old historical data)을 통해 학습하는 방법 
    - 과거 데이터 : 이전 버전 에이전트, 인간의 설명에 의한 기록, 같은 에이전트가 여러 에피소드를 관찰 등을 통해 얻은 데이터
    
- 학습하는 정책과 행동하는 정책이 반드시 같지 않아도 됨 
    
part2, part3에서 더 자세히 다룸 
    
__________________________________________________________________________________________________________________________________________
    
### 이번 장에서 살펴 볼 Cross-entropy 
#### - model-free, policy-based, on-policy 
- 환경에 대한 어떤 모델도 생성하지 않고 매 step마다 에이전트가 해야 할 행동만 알려줌 
- 에이전트의 정책을 근사화 
- 환경으로부터 얻을 수 있는 새로운(fresh) 데이터 사용 

# <font color="blue"> Practical cross-entropy <font>
    
#### cross-entropy에 대한 설명은 실용적인 (practical) 부분과 이론적인 (theorerical) 부분으로 나눔 
- 실용적 설명 : 본질에 대해 직관적으로 설명
- 이론적 설명 : 왜 작동하는지, 어떤 일이 일어나는지 더 정교하게 설명 

## * High-level approach to RL 

#### - 강화학습에서 가장 중심되고 까다로운 것은 agent 
- agent는 환경과의 소통을 위해 총보상을 최대한 축적하려 함 

#### - 실제로, 일반적인 기계학습 (ML) 접근법에 따라, agent의 복잡한 부분은 비선형함수로 대체할 수 있음 
- 비선형 함수(nonlinear trainable function)는 agent의 입력(환경으로부터의 관찰)을 어떠한 출력으로 사상(mapping)  
- 이러한 함수의 출력에 대한 세부사항은 방법에 따라 달라짐 ((ex) value-based / policy-based)

#### - 
<img src = "./image/4/rlml.png" width=400>


- 여기서의 Cross-entropy는 policy-based
- 비선형함수 (신경망)은 정책을 생산 
    - 정책 : 관찰마다 agent가 수행해야 할 액션을 결정 
    
#### - 실제로, 보통 정책은 액션의 확률 분포로 표현됨 
- 분류 문제와 매우 비슷해짐 
<img src = "./image/4/nn.jpg" width=400>
- 분류하는 클래스의 개수 = 수행 할 수 있는 액션의 개수

#### - 이러한 추상적인 개념을 통해 agent를 간단히 만들 수 있음 
 1. 환경으로부터의 관찰 (input)을 네트워크로 전달 
 2. 액션에 대한 확률분포 (output) 를 얻음  
 3. 확률분포를 통해 랜덤 샘플링을 하여 수행할 액션을 얻음 
      - 랜덤 샘플링은 에이전트에 무작위성을 더해줌  
 4. 환경에 액션을 수행하여 마지막 액션에 대한 다음 환경과 보상을 얻음 
 5. 이 과정을 반복 
 
## * Cross-entropy 수행 과정 
#### - agent의 생애 동안 경험은 에피소드로 존재함 
- 모든 에피소드는 'agent가 환경으로부터 얻는 관찰, 수행할 수 있는 행동, 그러한 액션에 대한 보상' 의 연속


- agent가 이러한 에피소드를 여러개 수행한다면 모든 에피소드마다 우리는 agent가 얻은 총보상을 계산할 수 있음 


- 이 총보상은 할인될 수도 있고 안될수도 있음
- 하지만 간단하게 하기 위해 (할인 계수 = 1)이라 가정 
    - 총보상은 모든 보상을 단순히 더한 것이 됨 
 
 
- 이 총보상은 에피소드가 agent에게 얼마나 좋은지를 보여줌 (agent가 에피소드를 얼마나 잘 수행했는지) 

#### - 예시 
<img src = "./image/4/episode.png">


- 4개의 에피소드 


- 각 에피소드는 다른 값의 oi(관찰값)와 ai(액션)와 ri(보상)를 가짐 


- 각각의 셀은 에피소드 내에서의 agent의 step을 표현 


- agent는 랜덤하게 액션을 선택하기 때문에 어떤 에피소드는 다른 에피소드들 보다 나을 수 있음 


#### - cross-entorpy의 주요점은 나쁜 에피소드는 버리고 좋은 에피소드에 대해서 학습하는 것 


- 방법 
     1. 현재 모델과 환경에 대해 n 개의 에피소드를 수행 
     
     
     2. 매 에피소드마다 총 보상을 계산하고 보상경계를 결정
         - 보통 총보상의 백분위수(50, 70)을 사용 
            
     3. 보상경계보다 보상이 낮은 에피소드를 버림 
     
     4. 남은 '엘리트' 에피소드의 관찰을 입력으로, 관련된 액션을 출력(target)으로 사용
     
     5. 결과에 만족할 때까지 위 과정을 반복 
    
    
  
- 신경망은 더 큰 보상을 주는 행동을 반복하는 방법을 학습하고 지속적으로 보상경계는 높아질 것  
- 단순성에도 불구하고 이 방법은 간단한 환경에서 잘 작동 
- 구현도 쉽고, 하이퍼파라미터의 변화에도 꽤 굳건(robust)
